set parameters

In [1]:
import argparse
import os
from util import util
import torch
import models
import data
import time
from options.train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer
import torch.nn as nn
import easydict

In [2]:
option_json=easydict.EasyDict({#BASE OPTIONS
    "name":"experiment_name",
    "gpu_ids":[0,1],
    "checkpoints_dir":"./checkpoints",
    "model":"cycle_gan",
    "input_nc":1,
    "output_nc":1,
    "ngf":64,
    "ndf":64,
    "netD":"basic",
    "netG":"resnet_9blocks",
    "n_layers_D":3,
    "norm":"batch",
    "init_type":"normal",
    "init_gain":0.02,
    "no_dropout":"store_true",
    "dataset_mode":"unaligned",
    "direction":"AtoB",
    "serial_batches":"store_true",
    "num_threads":4,
    "batch_size":1,
    "load_size":256,
    "crop_size":256,
    "max_dataset_size":float('inf'),
    "preprocess":"resize_and_crop",
    "no_flip":"store_true",
    "display_winsize":"256",
    "epoch":"latest",
    "load_iter":0,
    "verbose":"store_true",
    "suffix":"", #TRAIN OPTIONS
    "isTrain":True,
    "display_freq":200,
    "display_ncols":4,
    "display_id":1,
    "display_server":"http://localhost",
    "display_env":"main",
    "display_port":8097,
    "update_html_freq":1000,
    "print_freq":100,
    "no_html":"store_true",
    "save_latest_freq":400,
    "save_epoch_freq":1,
    "save_by_iter":"store_true",
    "continue_train":"store_true",
    "epoch_count":1,
    "phase_train":"train",
    "dataroot":'./processed',
    "niter":100,
    "niter_decay":100,
    "beta1":0.5,
    "lr":0.0002,
    "gan_mode":'vanilla',
    "pool_size":50,
    "lr_policy":"lambda",
    'lambda_A':10.0,
    'lambda_B':10.0, 
    'lambda_identity':0.5, 
    "lr_decay_iters":50,#TEST OPTIONS
    "ntest":float("inf"),
    "results_dir":'./results/',
    "aspect_ratio":1.0,
    "phase_test":'test',
    "eval":"store_true",
    "num_test":50
})


In [3]:
#train_opt = TrainOptions().parse(option_json)   # get training options

dataset = create_dataset(option_json)  # create a dataset given opt.dataset_mode and other options
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

model = create_model(option_json)      # create a model given opt.model and other options
model = nn.DataParallel(model)    
model.module.setup(option_json)               # regular setup: load and print networks; create schedulers
visualizer = Visualizer(option_json)   # create a visualizer that display/save images and plots
total_iters = 0                # the total number of training iterations


dataset [UnalignedDataset] was created
The number of training images = 2423
initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
model [CycleGANModel] was created
loading the model from ./checkpoints/experiment_name/latest_net_G_A.pth
loading the model from ./checkpoints/experiment_name/latest_net_G_B.pth
loading the model from ./checkpoints/experiment_name/latest_net_D_A.pth
loading the model from ./checkpoints/experiment_name/latest_net_D_B.pth
---------- Networks initialized -------------
DataParallel(
  (module): ResnetGenerator(
    (model): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), bias=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1,

In [4]:
for epoch in range(option_json.epoch_count, option_json.niter + option_json.niter_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    dataset = create_dataset(option_json)  # create a dataset given opt.dataset_mode and other options
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch

    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % option_json.print_freq == 0:
            t_data = iter_start_time - iter_data_time
        visualizer.reset()
        total_iters += option_json.batch_size
        epoch_iter += option_json.batch_size
        model.module.set_input(data)         # unpack data from dataset and apply preprocessing
        model.module.optimize_parameters()   # calculate loss functions, get gradients, update network weights

        if total_iters % option_json.display_freq == 0:   # display images on visdom and save images to a HTML file
            save_result = total_iters % option_json.update_html_freq == 0
            model.module.compute_visuals()
            visualizer.display_current_results(model.module.get_current_visuals(), epoch, save_result)

        if total_iters % option_json.print_freq == 0:    # print training losses and save logging information to the disk
            losses = model.module.get_current_losses()
            t_comp = (time.time() - iter_start_time) / option_json.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            if option_json.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

        if total_iters % option_json.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
            save_suffix = 'iter_%d' % total_iters if option_json.save_by_iter else 'latest'
            model.module.save_networks(save_suffix)

        iter_data_time = time.time()
    if epoch % option_json.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
        model.module.save_networks('latest')
        model.module.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, option_json.niter + option_json.niter_decay, time.time() - epoch_start_time))
    model.module.update_learning_rate()                     # update learning rates at the end of every epoch.


dataset [UnalignedDataset] was created
(epoch: 1, iters: 100, time: 0.243, data: 0.117) D_A: 0.584 G_A: 1.335 cycle_A: 0.574 idt_A: 0.093 D_B: 0.217 G_B: 2.257 cycle_B: 0.404 idt_B: 0.143 
(epoch: 1, iters: 200, time: 0.400, data: 0.001) D_A: 0.104 G_A: 1.403 cycle_A: 0.668 idt_A: 0.118 D_B: 0.171 G_B: 2.190 cycle_B: 0.830 idt_B: 0.175 
(epoch: 1, iters: 300, time: 0.243, data: 0.001) D_A: 0.103 G_A: 1.885 cycle_A: 0.341 idt_A: 0.066 D_B: 0.238 G_B: 2.475 cycle_B: 0.372 idt_B: 0.104 
(epoch: 1, iters: 400, time: 0.333, data: 0.001) D_A: 0.092 G_A: 0.973 cycle_A: 0.268 idt_A: 0.069 D_B: 0.183 G_B: 0.527 cycle_B: 0.279 idt_B: 0.095 
saving the latest model (epoch 1, total_iters 400)
(epoch: 1, iters: 500, time: 0.243, data: 0.001) D_A: 0.420 G_A: 3.965 cycle_A: 0.600 idt_A: 0.105 D_B: 0.808 G_B: 3.870 cycle_B: 0.643 idt_B: 0.178 


Process Process-4:
Process Process-3:
Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/vasant/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/vasant/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/vasant/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/vasant/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/vasant/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/vasant/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/vasant/anaconda3/lib/python3.6/multiprocessing

KeyboardInterrupt: 